In [18]:
from pathlib import Path
import re
from typing import Any, Dict, List

# Use the provided meeting notes. You can also point MARKDOWN_PATH to a file in Colab.
RAW_MARKDOWN = """# Product Team Sync - May 15, 2023

## Attendees
- Sarah Chen (Product Lead)
- Mike Johnson (Engineering)
- Anna Smith (Design)
- David Park (QA)

## Agenda

### 1. Sprint Review
* Completed Features
  * User authentication flow
  * Dashboard redesign
  * Performance optimization
    * Reduced load time by 40%
    * Implemented caching solution
* Pending Items
  * Mobile responsive fixes
  * Beta testing feedback integration

### 2. Current Challenges
* Resource constraints in QA team
* Third-party API integration delays
* User feedback on new UI
  * Navigation confusion
  * Color contrast issues

### 3. Next Sprint Planning
* Priority Features
  * Payment gateway integration
  * User profile enhancement
  * Analytics dashboard
* Technical Debt
  * Code refactoring
  * Documentation updates

## Action Items
- [ ] @sarah: Finalize Q3 roadmap by Friday
- [ ] @mike: Schedule technical review for payment integration
- [ ] @anna: Share updated design system documentation
- [ ] @david: Prepare QA resource allocation proposal

## Next Steps
* Schedule individual team reviews
* Update sprint board
* Share meeting summary with stakeholders

## Notes
* Next sync scheduled for May 22, 2023
* Platform demo for stakeholders on May 25
* Remember to update JIRA tickets

---
Meeting recorded by: Sarah Chen
Duration: 45 minutes
"""

# Optional: set to a path like "/content/process.md" to load markdown from file
MARKDOWN_PATH = None
markdown_text = Path(MARKDOWN_PATH).read_text() if MARKDOWN_PATH else RAW_MARKDOWN
print(markdown_text[:300] + "...\n\nLoaded markdown characters:", len(markdown_text))


# Product Team Sync - May 15, 2023

## Attendees
- Sarah Chen (Product Lead)
- Mike Johnson (Engineering)
- Anna Smith (Design)
- David Park (QA)

## Agenda

### 1. Sprint Review
* Completed Features
  * User authentication flow
  * Dashboard redesign
  * Performance optimization
    * Reduced load ...

Loaded markdown characters: 1333


In [19]:
from google.colab import auth
from google.auth import default
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

DOCS_SCOPE = ["https://www.googleapis.com/auth/documents"]

try:
    auth.authenticate_user()
    creds, _ = default(scopes=DOCS_SCOPE)
    docs_service = build("docs", "v1", credentials=creds)
    print("✔ Authenticated and Docs service ready")
except Exception as exc:  # broad to surface any auth issues in Colab
    raise RuntimeError(f"Authentication failed: {exc}")


✔ Authenticated and Docs service ready


In [20]:
Element = Dict[str, Any]


def parse_markdown(md_text: str) -> List[Element]:
    """Parse markdown lines into a structured list for Docs requests.

    Recognizes:
    - Headings (#, ##, ###)
    - Bullets (* or -) with nesting via indentation (2 spaces per level)
    - Checkboxes (- [ ])
    - Footer lines after a markdown horizontal rule (---)
    - Plain paragraphs
    """
    lines = md_text.strip().splitlines()
    elements: List[Element] = []
    in_footer = False

    for raw_line in lines:
        line = raw_line.rstrip("\n")
        if not line.strip():
            continue  # skip blank lines, Docs paragraphs will separate via \n
        if line.strip() == "---":
            in_footer = True
            continue

        if in_footer:
            elements.append({"type": "footer", "text": line.strip()})
            continue

        heading_match = re.match(r"^(#+)\s+(.*)", line)
        if heading_match:
            level = min(len(heading_match.group(1)), 6)
            text = heading_match.group(2).strip()
            elements.append({"type": "heading", "level": level, "text": text})
            continue

        checkbox_match = re.match(r"^(\s*)-\s*\[\s*\]\s+(.*)", line)
        if checkbox_match:
            indent = checkbox_match.group(1)
            level = len(indent) // 2
            text = checkbox_match.group(2).strip()
            elements.append({"type": "checkbox", "level": level, "text": text})
            continue

        bullet_match = re.match(r"^(\s*)[-*]\s+(.*)", line)
        if bullet_match:
            indent = bullet_match.group(1)
            level = len(indent) // 2
            text = bullet_match.group(2).strip()
            elements.append({"type": "bullet", "level": level, "text": text})
            continue

        # Fallback plain paragraph
        elements.append({"type": "paragraph", "text": line.strip()})

    return elements


# Quick sanity check
parsed_preview = parse_markdown(markdown_text)
print(f"Parsed elements: {len(parsed_preview)}")
print(parsed_preview[:5])


Parsed elements: 46
[{'type': 'heading', 'level': 1, 'text': 'Product Team Sync - May 15, 2023'}, {'type': 'heading', 'level': 2, 'text': 'Attendees'}, {'type': 'bullet', 'level': 0, 'text': 'Sarah Chen (Product Lead)'}, {'type': 'bullet', 'level': 0, 'text': 'Mike Johnson (Engineering)'}, {'type': 'bullet', 'level': 0, 'text': 'Anna Smith (Design)'}]


In [21]:
MentionStyle = {
    "bold": True,
    "foregroundColor": {"color": {"rgbColor": {"red": 0.05, "green": 0.35, "blue": 0.80}}},
}
FOOTER_STYLE = {
    "bold": False,
    "italic": True,
    "foregroundColor": {"color": {"rgbColor": {"red": 0.35, "green": 0.35, "blue": 0.35}}},
}


def style_mentions(requests: List[Dict[str, Any]], base_index: int, text: str) -> None:
    """Apply bold/color styling to @mentions within the inserted text."""
    for match in re.finditer(r"@\w+", text):
        start = base_index + match.start()
        end = base_index + match.end()
        requests.append(
            {
                "updateTextStyle": {
                    "range": {"startIndex": start, "endIndex": end},
                    "textStyle": MentionStyle,
                    "fields": "bold,foregroundColor",
                }
            }
        )


def insert_paragraph(requests: List[Dict[str, Any]], cursor: int, text: str) -> int:
    """Insert text plus a newline and return the new cursor position."""
    requests.append({"insertText": {"location": {"index": cursor}, "text": text + "\n"}})
    return cursor + len(text) + 1


def add_heading(requests: List[Dict[str, Any]], cursor: int, text: str, level: int) -> int:
    start = cursor
    end = insert_paragraph(requests, cursor, text)
    requests.append(
        {
            "updateParagraphStyle": {
                "range": {"startIndex": start, "endIndex": end},
                "paragraphStyle": {"namedStyleType": f"HEADING_{min(level, 6)}"},
                "fields": "namedStyleType",
            }
        }
    )
    style_mentions(requests, start, text)
    return end


def add_bullet(
    requests: List[Dict[str, Any]], cursor: int, text: str, level: int, preset: str
) -> int:
    start = cursor
    end = insert_paragraph(requests, cursor, text)
    requests.append(
        {
            "createParagraphBullets": {
                "range": {"startIndex": start, "endIndex": end},
                "bulletPreset": preset,
                "nestingLevel": level,
            }
        }
    )
    style_mentions(requests, start, text)
    return end


def add_paragraph(requests: List[Dict[str, Any]], cursor: int, text: str) -> int:
    start = cursor
    end = insert_paragraph(requests, cursor, text)
    style_mentions(requests, start, text)
    return end


def add_footer(requests: List[Dict[str, Any]], cursor: int, text: str) -> int:
    start = cursor
    end = insert_paragraph(requests, cursor, text)
    requests.append(
        {
            "updateTextStyle": {
                "range": {"startIndex": start, "endIndex": end},
                "textStyle": FOOTER_STYLE,
                "fields": "italic,foregroundColor",
            }
        }
    )
    return end


In [32]:
def build_google_doc_from_markdown(
    service, markdown_text: str, document_title: str = "Product Team Sync"
) -> str:
    """Create a Google Doc and populate it based on the parsed markdown."""
    try:
        new_doc = service.documents().create(body={"title": document_title}).execute()
    except HttpError as http_err:
        raise RuntimeError(f"Unable to create document: {http_err}")

    doc_id = new_doc.get("documentId")
    elements = parse_markdown(markdown_text)
    requests: List[Dict[str, Any]] = []
    cursor = 1  # start of the document

    BULLET_PRESET = "BULLET_DISC_CIRCLE_SQUARE"
    CHECKBOX_PRESET = "BULLET_CHECKBOX"

    def add_bullet(requests, cursor, text, level, preset):
        start = cursor
        end = insert_paragraph(requests, cursor, text)
        requests.append(
            {
                "createParagraphBullets": {
                    "range": {"startIndex": start, "endIndex": end},
                    "bulletPreset": preset
                      }
            }
        )
        indent_pt = 18 * level  # ~0.25in per level
        requests.append(
        {
            "updateParagraphStyle": {
                "range": {"startIndex": start, "endIndex": end},
                "paragraphStyle": {
                    "indentStart": {"magnitude": indent_pt, "unit": "PT"}
                },
                "fields": "indentStart",
            }
        }
    )
        style_mentions(requests, start, text)
        return end


    for element in elements:
        etype = element["type"]
        if etype == "heading":
            cursor = add_heading(requests, cursor, element["text"], element["level"])
        elif etype == "bullet":
            cursor = add_bullet(requests, cursor, element["text"], element["level"], BULLET_PRESET)
        elif etype == "checkbox":
            cursor = add_bullet(requests, cursor, element["text"], element["level"], CHECKBOX_PRESET)
        elif etype == "footer":
            cursor = add_footer(requests, cursor, element["text"])
        else:  # plain paragraph
            cursor = add_paragraph(requests, cursor, element["text"])

    try:
        service.documents().batchUpdate(documentId=doc_id, body={"requests": requests}).execute()
    except HttpError as http_err:
        raise RuntimeError(f"Failed to write to document: {http_err}")

    print(f"✅ Created Google Doc: https://docs.google.com/document/d/{doc_id}/edit")
    print("(You may need to refresh if the doc doesn't immediately show styles.)")
    return doc_id

In [39]:
DOC_TITLE = "Product Team Sync"

try:
    created_doc_id = build_google_doc_from_markdown(docs_service, markdown_text, document_title=DOC_TITLE)
    print("Document ID:", created_doc_id)
except Exception as exc:
    print("❌ Failed to build document:", exc)


❌ Failed to build document: Failed to write to document: <HttpError 400 when requesting https://docs.googleapis.com/v1/documents/1hy7F3JnaeEi86RrG4N0k6IDrGGBeIJoj2erc9GCoYpA:batchUpdate?alt=json returned "Invalid value at 'requests[0].create_footer.type' (type.googleapis.com/google.apps.docs.v1.HeaderFooterType), "DEFAULT_FOOTER"". Details: "[{'@type': 'type.googleapis.com/google.rpc.BadRequest', 'fieldViolations': [{'field': 'requests[0].create_footer.type', 'description': 'Invalid value at \'requests[0].create_footer.type\' (type.googleapis.com/google.apps.docs.v1.HeaderFooterType), "DEFAULT_FOOTER"'}]}]">
